In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import os
import logging

In [2]:
#os.chdir('..')
os.chdir('C:/Users/BC966HL/OneDrive - EY/4. Mestrado/Dev/')
os.getcwd()

'C:\\Users\\BC966HL\\OneDrive - EY\\4. Mestrado\\Dev'

In [3]:
from Coding.etl.data_normalization import data_norm
from Coding.etl.input_parameters import (recipe_def, create_cost_column, input_lots, target_volume, model_files)
from Coding.model.minimize import (model_input_read, constraints_bounds, cost_obj_f, minimize_round, run_model, model_results)

In [4]:
# Reading CQI CSV file as input, normalizing data and creating cleaned_cqi_file.csv with no-NaN rows and consistent coffee sample parameters
data_norm()

# recipe_def function to read the cleaned_cqi_file.csv and extract a random row to be used as target for the model. It will return a single row dataframe with unusable LOT_AVAILABILITY_KG field and the normalized dataframe with the selected row removed
df_recipe, df_norm = recipe_def()

# create_cost_column gets the normalized dataframe without the recipe row and create a random cost per kg in BRL to the lot on the row, the model will seek to minimize this final cost attending to the other parameters
df_norm_cost = create_cost_column(df_norm)

# input_lots will get the normalized dataframe without the recipe row and with the cost column and extracts from the n available rows a lot_limit (default = 10) sized dataframe which will be used as available lots to the model work on
df_lots_available = input_lots(df_norm_cost, return_df_norm_wout_lots = False, lot_limit = 10)

# df_recipe_volume simulate the batch volume
# if simulate_broken_volume is set to True, it will create a volume higher than the available lots can provide together, reaching from the volume sum to broken_perc*volume sum, the broken_perc is set to 1.5 as default
# if is set to False, will get from the lots available volume sum * low_perc to volume sum * high perc, which are set as default to 0.2 and 0.8 respectively
df_recipe_volume = target_volume(df_recipe, df_lots_available, simulate_broken_volume = False)

# model_files export the recipe and available lots datasets to /Datasets/model as input_recipe.csv and input_lots_available.csv with ';' as CSV separator
model_files(df_recipe_volume, df_lots_available)

In [5]:
df_recipe, df_available_lots = model_input_read()

constraints, bnd = constraints_bounds(df_recipe, df_available_lots, ub = 0.7)

In [6]:
try:
    model_output = run_model(df_recipe, df_available_lots, constraints, bnd, times = 100)

except:
   logging.warning('Model could not find a solution within range and iterations')

In [7]:
print(model_results(df_recipe, df_available_lots, model_output))

(   LOT_AVAILABILITY_KG  MODEL_RESULT_ARRAY  ALLOCATION_TEST
0                250.0                   0             True
1                300.0                 300             True
2                 10.0                   9             True
3                 26.0                   0             True
4                  2.0                   0             True
5                  2.0                   1             True
6              18975.0               18975             True
7                  4.0                   0             True
8                600.0                   0             True
9              19200.0                6628             True,       AROMA    FLAVOR  AFTERTASTE   ACIDITY      BODY   BALANCE  MOISTURE
0  0.432206  0.062266    0.364306  0.079772  0.093175  0.342822  0.000005,    AROMA  FLAVOR  AFTERTASTE  ACIDITY  BODY  BALANCE  MOISTURE
0   7.67    7.42         7.5     7.42  7.58      7.5      0.11,       AROMA    FLAVOR  AFTERTASTE   ACIDITY      BODY   BALANC